## Часть 3. Классификация текстов [40/100]

Сформулируем для простоты задачу бинарной классификации: будем классифицировать на два класса, то есть, различать резко отрицательные отзывы (с оценкой 1) и положительные отзывы (с оценкой 5). 

1.  Составьте обучающее и тестовое множество: выберите из всего набора данных N1 отзывов с оценкой 1 и N2 отзывов с оценкой 5 (значение N1 и N2 – на ваше усмотрение). Используйте ```sklearn.model_selection.train_test_split``` для разделения множества отобранных документов на обучающее и тестовое. 
2. Используйте любой известный вам алгоритм классификации текстов для решения задачи и получите baseline. Сравните разные варианты векторизации текста: использование только униграм, пар или троек слов или с использованием символьных $n$-грам. 
3. Сравните, как изменяется качество решения задачи при использовании скрытых тем в качестве признаков:
* 1-ый вариант: $tf-idf$ преобразование (```sklearn.feature_extraction.text.TfidfTransformer```) и сингулярное разложение (оно же – латентый семантический анализ) (```sklearn.decomposition.TruncatedSVD```), 
* 2-ой вариант: тематические модели LDA (```sklearn.decomposition.LatentDirichletAllocation```). 
 

In [2]:
import json

import bz2
import regex
from tqdm import tqdm
from scipy import sparse
import pandas as pd
import numpy as np



import nltk
from pymystem3 import Mystem
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem.snowball import RussianStemmer
from gensim.corpora import *
from gensim.models import  *

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from gensim.sklearn_api import LsiTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation as LDA

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV


from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

%matplotlib inline
%pylab inline



Populating the interactive namespace from numpy and matplotlib


In [3]:


responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [02:51, 1169.50it/s]


#### Датасет


In [4]:
train_list = []
for resp in responses:
#     print(list(resp.values()))
    train_list.append(list(resp.values()))
train = pd.DataFrame(columns = list(responses[99].keys()), data = train_list)
del train_list

In [5]:
train.head()

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
0,г. Москва,False,Жалоба,0,лицензия № 2562,uhnov1,Бинбанк,2015-06-08 12:50:54,Добрый день! Я не являюсь клиентом банка и пор...,NaN
1,г. Новосибирск,False,Не могу пользоваться услугой Сбербанк он-лайн,0,лицензия № 1481,Foryou,Сбербанк России,2015-06-08 11:09:57,Доброго дня! Являюсь держателем зарплатной кар...,NaN
2,г. Москва,False,Двойное списание за один товар.,1,лицензия № 2562,Vladimir84,Бинбанк,2015-06-05 20:14:28,Здравствуйте! Дублирую свое заявление от 03.0...,NaN
3,г. Ставрополь,False,Меняют проценты комиссии не предупредив и не ...,2,лицензия № 1481,643609,Сбербанк России,2015-06-05 13:51:01,Добрый день!! Я открыл расчетный счет в СберБа...,NaN
4,г. Челябинск,False,Верните денежные средства за страховку,1,лицензия № 2766,anfisa-2003,ОТП Банк,2015-06-05 10:58:12,"04.03.2015 г. взяла кредит в вашем банке, заяв...",NaN


In [6]:
m = Mystem()
regex = re.compile("[А-Яа-я]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""

def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

def stemming(text, stemmer = RussianStemmer()):
    try:
        return " ".join([stemmer.stem(w) for w in text.split()])
    except:
        return " "

#### Очистка
- оставляются только слова
- лемметизация
- стемминг


In [7]:
%%time
data = train[(train['rating_grade'] == 5)|(train['rating_grade'] == 1)]

data['text_words'] = data.apply(lambda row: words_only(row['text']), axis = 1)
data['text_lemmas'] = data.apply(lambda row: lemmatize(row['text_words']), axis = 1)
data['text_stemm'] = data.apply(lambda row: stemming(row['text_lemmas']), axis = 1)

/home/vvs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/vvs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


CPU times: user 19min 36s, sys: 7.1 s, total: 19min 43s
Wall time: 37min 32s


/home/vvs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


#### Пример
- До очистки 


In [8]:

data['text'][19]

'Открыт вклад и счет в USD. Плюс к этому есть зарплатная карта, в рублях, само собой. Сегодня пришел в указанное отделение с целью пополнить долларовый счёт на 700 USD.\xa0Дал операционисту паспорт, зарплатную карту (т.к. на окошке написано "приготовьте карту для подтверждения операции" или что-то подобное и в прошлый раз у меня ее потребовали) и сказал, что нужно положить деньги на ДОЛЛАРОВЫЙ счет.\xa0Операционист всё взяла, что-то делала-крутила-вертела, вставила карту в терминал, сказала "введите пин", я ввёл пин, получил в ответ чек, где было написано, что доллары были внесены.... на счёт КАРТЫ! в РУБЛЯХ! Вопрос банку №1, риторический:  Я не понимаю, кем нужно быть, чтобы сознательно проводить такие операции??? за углом, меньше чем через квартал, курс приёма валюты выше почти на рубль! Если я действительно хотел совершить такую "хитрую" операцию, мне было выгоднее сделать 100 шагов и "заработать" на этом около 700 рублей, после чего просто внести рубли на счёт карты в банкомате! Да

#### Пример
- после очистки


In [18]:

data.head()
data['text_stemm'][19]

'открыва вклад и счет в плюс к эт быт зарплатн карт в рубл сам себ сегодн приход в указа отделен с цел пополня долларов сч т на дава операционист паспорт зарплатн карт т к на окошк написа приготавлива карт для подтвержден операц ил что то подобн и в прошл раз у я он потребова и сказа что нужн полага деньг на долларов счет операционист вс взят что то дела крут вертел вставля карт в термина сказа ввод пин я вв л пин получа в ответ чек где быт написа что доллар быт внос на сч т карт в рубл вопрос банк риторическ я не понима кто нужн быт чтоб сознательн провожа так операц за угол мал чем через кварта курс при ма валют высок почт на рубл есл я действительн хотет соверша так хитр операц я быт выгодн сдела шаг и зарабатыва на эт окол рубл посл что прост внос рубл на сч т карт в банкомат дал посл долг окол два час выяснен отношен с операционист и видим е начальник их попытк отменя перв транзакц и т д и т п мы сход на то что я компенсирова курсов разниц межд зачислен доллар на рубл ву карт руб 

#### разделение на тестовую и обучающую части


In [19]:

X_train, X_test, y_train, y_test = train_test_split(
    data['text_stemm'], data['rating_grade'], test_size=0.33, random_state=42)

#### создание словаря

In [20]:
%%time


texts = [X_train.iloc[i].split() for i in range(len(X_train))]
dictionary = Dictionary(texts)



CPU times: user 1min 24s, sys: 2.63 s, total: 1min 27s
Wall time: 1min 21s


### Проба последовательной обработки  и классификации
 -  TfidfVectorizer
 -  Lsi (TruncatedSVD)
 -  LogisticRegression

In [37]:
%%time

clf = LogisticRegression(penalty='l2', C=0.1)

pipe = Pipeline([('tfidf', TfidfVectorizer()), ('tm', TruncatedSVD()), ('classifier', clf)])

CPU times: user 294 µs, sys: 4 µs, total: 298 µs
Wall time: 305 µs


In [38]:
%%time
score = pipe.fit(X_train, y_train).score(X_test, y_test)

CPU times: user 16.4 s, sys: 388 ms, total: 16.8 s
Wall time: 16.1 s


#### Accuracy

In [39]:
score 

0.764846532962475

#### Проба улучшить качество работы заменой классификатора на GradientBoostingClassifier()

In [75]:
%%time

clf = GradientBoostingClassifier()

pipe = Pipeline([
    ('tfidf', TfidfVectorizer()), 
    ('tm', TruncatedSVD()), 
    ('classifier', clf)
])

CPU times: user 206 µs, sys: 2 µs, total: 208 µs
Wall time: 217 µs


In [76]:
%%time
score = pipe.fit(X_train, y_train).score(X_test, y_test)

CPU times: user 20.1 s, sys: 248 ms, total: 20.3 s
Wall time: 19.6 s


In [77]:
score 

0.7837798272580881

#### Метрика стала немного лучше

In [79]:
stop_words_russian = ["и", "в", "во",  "что", "он", "на", "я", "с", "со", "как", "а", "то", "все", "она", "так", 
                      "его", "но", "да", "ты", "к", "у", "же", "вы", "за", "бы", "по", "только", "ее", "мне", "было", 
                      "вот", "от", "меня", "еще", "о", "из", "ему", "теперь", "когда", "даже", "ну", "вдруг", 
                      "ли", "если", "уже", "или", "ни", "быть", "был", "него", "до", "вас", "нибудь", "опять", "уж", 
                      "вам", "ведь", "там", "потом", "себя", "ничего", "ей", "может", "они", "тут", "где", "есть", "надо", 
                      "ней", "для", "мы", "тебя", "их", "чем", "была", "сам", "чтоб", "без", "будто", "чего", "раз", 
                      "тоже", "себе", "под", "будет", "ж", "тогда", "кто", "этот", "того", "потому", "этого", "какой", 
                      "совсем", "ним", "здесь", "этом", "один", "почти", "мой", "тем", "чтобы", "нее", "сейчас", "были", 
                      "куда", "зачем", "всех", "никогда", "можно", "при", "наконец", "два", "об", "другой", "хоть", "после", 
                      "над", "больше", "тот", "через", "эти", "нас", "про", "всего", "них", "какая", "много", "разве", "три", 
                      "эту", "моя", "впрочем", "хорошо", "свою", "этой", "перед", "иногда", "лучше", "чуть", "том", "нельзя", 
                      "такой", "им", "более", "всегда", "конечно", "всю", "между", "при", "однако", "это", "всё", "весь",
                      "обо", "ваш" , " " , "  ", " - ", "-", "   "]

In [85]:
boost_param_grid = {
    "tm": ["passthrough", TruncatedSVD(5),TruncatedSVD(10),TruncatedSVD(15), TruncatedSVD(20),TruncatedSVD(25)],
    "tfidf__analyzer": ["word", "char"],
    "tfidf__smooth_idf": [True, False],
    "tfidf__ngram_range": [(1, 1), (1, 2)],
    "tfidf__use_idf": [True, False],
    "tfidf__stop_words": [None, stop_words_russian],
    "classifier__loss":["deviance"],
    "classifier__learning_rate": [0.01, 0.05, 0.1, 0.2],
    "classifier__min_samples_split": np.linspace(0.1, 0.5, 12),
    "classifier__min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "classifier__max_depth":[3,5,8],
    "classifier__max_features":["log2","sqrt"],
    "classifier__criterion": ["friedman_mse",  "mae"],
    "classifier__subsample":[0.5, 0.8, 0.9, 1.0],
    "classifier__n_estimators":[10]
}

#### Поиск оптимальных параметров алгоритма обработки с помощью RandomizedSearchCV

In [86]:
%%time
search = RandomizedSearchCV(pipe, param_distributions=boost_param_grid, verbose=8,n_jobs = -1)
search.fit(X_train, y_train)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 20.2min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 24.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 24.5min finished


CPU times: user 59.1 s, sys: 10.4 s, total: 1min 9s
Wall time: 25min 25s


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('tfidf',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                                                    

In [90]:
y_pred = search.predict(X_test)

In [91]:
print(f'acc {accuracy_score(y_test, y_pred)}')
print(f'f1_score {f1_score(y_test, y_pred)}')


acc 0.7634802127555751
f1_score 0.8656782596646806


#### Итоговые покзатели немного хуже. Скорее связано с тем, что было найдено более устойчивое решение

### Проба последовательной обработки  и классификации
 -  TfidfVectorizer
 -  LDA
 -  SVC

In [105]:
%%time


clf = SVC()

pipe2 = Pipeline([
    ('tfidf', TfidfVectorizer()), 
    ('tm', LDA()), 
    ('classifier', clf)
])

CPU times: user 384 µs, sys: 10 µs, total: 394 µs
Wall time: 1.61 ms


In [106]:
%%time
score = pipe.fit(X_train, y_train)

CPU times: user 21 s, sys: 1.15 s, total: 22.2 s
Wall time: 25.1 s


In [107]:
score 

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                                            learning_rate=0.1, loss='deviance',
                                            max_depth=3, max_features=None,
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
          

In [108]:
y_pred_linear_svc = score.predict(X_test)

In [109]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
print(f'acc {accuracy_score(y_test, y_pred_linear_svc)}')
print(f'f1_score {f1_score(y_test, y_pred_linear_svc)}')


acc 0.7835846386571024
f1_score 0.871154237238895


####  LDA + SVC дали  результат немного лучшего качества


In [ ]:
from IPython.display import Audio

Audio(url="https://wav-sounds.com/wp-content/uploads/2017/10/Herbert-06.wav",autoplay=True)

### Использованные материалы
https://medium.com/swlh/randomized-or-grid-search-with-pipeline-cheatsheet-719c72eda68

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC

https://www.kaggle.com/hatone/gradientboostingclassifier-with-gridsearchcv

https://towardsdatascience.com/boosting-showdown-scikit-learn-vs-xgboost-vs-lightgbm-vs-catboost-in-sentiment-classification-f7c7f46fd956